### === Analyzing a simple queueing network ===

In [ ]:
# How to define proper classes
# How to define proper functions

### === Create a queueing system according to kendall's notation ===

- https://de.wikipedia.org/wiki/Kendall-Notation

In queueing theory, a discipline within the mathematical theory of probability, Kendall's notation (or sometimes Kendall notation) is the standard system used to describe and classify a queueing node.

A/S/c/K/N/D

A: Arrival process
S: Service time distribution
c: Number of serive channels
K: Capacity of queue, or the maximum number of customers allowed in the queue.
N: Size of calling source. The size of the population from which the customers come
D: Service Discipline or Priority order that jobs in the queue, or waiting line, are served.

Note: When the final three parameters are not specified (e.g. M/M/1 queue), it is assumed K = ∞, N = ∞ and D = FIFO


### === Task: Design and simulate a (M/M/1/inf/inf/FIFO) queueing system ===

### === Wraping into one funciton ===

In [ ]:
import simpy
import random
import numpy as np
import pandas as pd

class Job():
    def __init__(self, job_id: int, job_type: str, failure_rate: float):
        self.id = job_id
        self.log = {}
        self.job_type = job_type
        self.failure_rate = failure_rate

    def __str__(self) -> str:
        return f'{self.id}({self.job_type})'

class Queue(simpy.Store): 
    def __init__(self, env, capacity=1, name='default', discipline='FIFO'):
        super().__init__(env, capacity)
        self.name = name
        self.log = {}
        self.state = {}
        self.discipline = discipline # is default of store object

def arrival_process(env, queue, systemLogs, distr='exponential', para=1.0):
    rng = np.random.default_rng(seed=42)
    job_id=0
    while True:
        job_id += 1
        rand_type = rng.choice(["A", "B", "C"], p=[0.1,0.1,0.8])
        ###################################
        #ex_lambda=para
        #t=random.expovariate(1/ex_lambda)
        t = rng.exponential(scale=para, size=1)[0]
        ###################################
        yield env.timeout(t)
        job = Job(job_id, rand_type, failure_rate=0.0)
        #print(f'[Time: {"%.3f" % env.now}] - Job {job.id} has been assigned to be be next processed in {queue.name}.')
        systemLogs.append({f'Job {job} has been assigned to be be next processed in {queue.name}.':env.now})
        job.log.update({f'Assinged to {queue.name}':env.now})
        yield queue.put(job)
        #print(f'[Time: {"%.3f" % env.now}] - Job {job.id} has been put to queue of {queue.name} and is waiting for service.')
        systemLogs.append({f'Job {job} has been put to queue of {queue.name} and is waiting for service.':env.now})
        job.log.update({f'Waiting in queue of {queue.name}':env.now})
        queue.log.update({f'Job {job} has accessed waiting queue in {queue.name}':env.now})


def service_process(env, queue_process, systemLogs, completedJobs, queue_next: dict=None, distr='exponential', para=1.0):
    rng = np.random.default_rng(seed=42)
    while True:
        job = yield queue_process.get()
        #print(f'[Time: {"%.3f" % env.now}] - Job {job.id} has been removed from queue and is being serviced in {queue_process.name}.')
        systemLogs.append({f'Job {job} has been removed from queue and is being serviced in {queue_process.name}.':env.now})
        job.log.update({f'Start servicing in QS {queue_process.name}':env.now})
        queue_process.log.update({f'Job {job} startet serving in {queue_process.name} ':env.now})
        ###################################
        #ex_lambda=para
        #t=random.expovariate(1/ex_lambda)
        #t = rng.exponential(scale=1.0, size=1)[0]
        t = rng.normal(loc=para, scale=0.2, size=1)[0]
        ###################################
        yield env.timeout(t)
        #print(f'[Time: {"%.3f" % env.now}] - Job {job.id} has finished being serviced in {queue_process.name}.')
        systemLogs.append({f'Job {job} has finished being serviced in {queue_process.name}.':env.now})
        job.log.update({f'Finish servicing in {queue_process.name}':env.now})
        queue_process.log.update({f'Job {job} finished serving in {queue_process.name} ':env.now})
        ### quality 
        qm = rng.uniform()
        if job.failure_rate > qm:
            systemLogs.append({f'Job {job} does not meet quality after {queue_process.name}.':env.now})
            job.log.update({f'Failed quality check after {queue_process.name}':env.now})
            queue_process.log.update({f'Job {job} failed quality check after {queue_process.name} ':env.now})
            yield queue_process.put(job)
            systemLogs.append({f'Job {job} has left {queue_process.name} and has been reinserted to waiting queue in {queue_process.name}.':env.now})
            job.log.update({f'Transferred to {queue_process.name}':env.now})
            queue_process.log.update({f'Job {job} has been reinsert into {queue_process.name} ':env.now})
        if queue_next == None:
            #print(f'[Time: {"%.3f" % env.now}] - Job {job.id} has been terminated from {queue_process.name}.')
            systemLogs.append({f'Job {job} has been terminated from {queue_process.name}.':env.now})
            job.log.update({f'Leaving {queue_process.name}':env.now})
            queue_process.log.update({f'Job {job} left {queue_process.name} ':env.now})
            completedJobs.append((env.now, job))
        else:
            #print(f'[Time: {"%.3f" % env.now}] - Job {job.id} is waiting after service in {queue_process.name} to access {queue_next.name}.')
            systemLogs.append({f'Job {job} is waiting after service in {queue_process.name} to access {queue_next[job.job_type].name}.':env.now})
            job.log.update({f'Completed at {queue_process.name}':env.now})
            job.log.update({f'Waiting for access to {queue_next[job.job_type].name}':env.now})
            yield queue_next[job.job_type].put(job)
            #print(f'[Time: {"%.3f" % env.now}] - Job {job.id} has left {queue_process.name} and been put to waiting queue in {queue_next.name}.')
            systemLogs.append({f'Job {job} has left {queue_process.name} and been put to waiting queue in {queue_next[job.job_type].name}.':env.now})
            job.log.update({f'Transferred to {queue_next[job.job_type].name}':env.now})
            queue_process.log.update({f'Job {job} left {queue_process.name} ':env.now})

def logging_process(env, queue):
    while True:
        #queue.state.update({env.now:f'Number of jobs in queue {len(queue.items)}.'})
        queue.state.update({round(env.now, 1): [str(i) for i in queue.items]})
        #queue.state.update({env.now: queue.items.copy()})
        yield env.timeout(0.1)
            
def run_mm1_experiment(runs):
    # Simulation
    completedJobs = []
    systemLogs = []
    env = simpy.Environment()
    # Entities
    queueingSystem_1 = Queue(env, capacity=np.inf, name='Queueing System 1')
    queueingSystem_2 = Queue(env, capacity=np.inf, name='Queueing System 2')
    queueingSystem_3 = Queue(env, capacity=np.inf, name='Queueing System 3')
    queueingSystem_4 = Queue(env, capacity=np.inf, name='Queueing System 4')
    queueingSystem_5 = Queue(env, capacity=np.inf, name='Queueing System 5')
    queueingSystem_6 = Queue(env, capacity=np.inf, name='Queueing System 6')

    # Ariival (system input) process
    arrival = env.process(arrival_process(env, queueingSystem_1, systemLogs))
    
    # tool processes (and logging)
    service1 = env.process(service_process(env, queueingSystem_1, systemLogs, completedJobs, queue_next = {'A': queueingSystem_2, 'B': queueingSystem_2, 'C': queueingSystem_5}))
    logging1 = env.process(logging_process(env, queueingSystem_1))
    
    service2 = env.process(service_process(env, queueingSystem_2, systemLogs, completedJobs, queue_next = dict.fromkeys(['A', 'B'], queueingSystem_3)))
    logging2 = env.process(logging_process(env, queueingSystem_2))
    
    service3 = env.process(service_process(env, queueingSystem_3, systemLogs, completedJobs, queue_next = dict.fromkeys(['A', 'B'], queueingSystem_4)))
    logging3 = env.process(logging_process(env, queueingSystem_3))
    
    service4 = env.process(service_process(env, queueingSystem_4, systemLogs, completedJobs)) # system output
    logging4 = env.process(logging_process(env, queueingSystem_4))
    
    service5 = env.process(service_process(env, queueingSystem_5, systemLogs, completedJobs, queue_next={'C': queueingSystem_6}))
    logging5 = env.process(logging_process(env, queueingSystem_5))
    
    service6 = env.process(service_process(env, queueingSystem_6, systemLogs, completedJobs, queue_next={'C': queueingSystem_4}))
    logging6 = env.process(logging_process(env, queueingSystem_6))

    # Run
    env.run(until=runs)
    
    results = (queueingSystem_1.state, queueingSystem_2.state, queueingSystem_3.state, queueingSystem_4.state, queueingSystem_5.state, queueingSystem_6.state)
    return results, completedJobs, systemLogs

In [ ]:
exp_res, completedJobs, systemLogs = run_mm1_experiment(runs=100)

In [ ]:
exp_res[0]

In [ ]:
systemLogs

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(figsize=(16,9), ncols=4, nrows=2)
for res, ax in zip(exp_res, axs.ravel()):
    #df = pd.DataFrame(res.items(), columns=['TimeStamp','Objects in Queue'])
    ax.plot(res.keys(), [len(r) for r in res.values()])

### === Results ===
- Expectation from Literature: The occupancy (p) is defined as the average arrival rate (lambda) divided by the average service rate (mu). For a stable system the average service rate should always be higher than the average arrival rate. Otherwise the queues would rapidly race towards infinity. Thus p should always be less than one.
- Hypothesis: Since we assume that mu=lambda=p=1, we expect an unsstable M/M/1-system where the avg queue lengh goes to infinity.
- Evidence: see simulated experiment data in graph :) 
- Links:
    - https://www.eventhelix.com/congestion-control/m-m-1/
    - https://en.wikipedia.org/wiki/M/M/1_queue
    - https://www.win.tue.nl/~iadan/que/h4.pdf

### === Follow Up ===

- Warum hängen die Trajektorien von der Art der Generatoren ab?
- Welche random number generators sind die richtigen?